In [2]:
import json
from collections import defaultdict
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

ImportError: No module named 'plotly'

## Loading a `match` document

A match is a document consisting of the following fields:

- **competitionId**: the identifier of the competition to which the match belongs to. It is a integer and refers to the field "wyId" of the competition document;
- **date** and **dateutc**: the former specifies date and time when the match starts in explicit format (e.g., May 20, 2018 at 8:45:00 PM GMT+2), the latter contains the same information but in the compact format YYYY-MM-DD hh:mm:ss;
- **duration**: the duration of the match. It can be "Regular" (matches of regular duration of 90 minutes + stoppage time), "ExtraTime" (matches with supplementary times, as it may happen for matches in continental or international competitions), or "Penalities" (matches which end at penalty kicks, as it may happen for continental or international competitions);
- **gameweek**: the week of the league, starting from the beginning of the league;
- **label**: contains the name of the two clubs and the result of the match (e.g., "Lazio - Internazionale, 2 - 3");
- **roundID**: indicates the match-day of the competition to which the match belongs to. During a competition for soccer clubs, each of the participating clubs plays against each of the other clubs twice, once at home and once away. The matches are organized in match-days: all the matches in match-day i are played before the matches in match-day i + 1, even tough some matches can be anticipated or postponed to facilitate players and clubs participating in Continental or Intercontinental competitions. During a competition for national teams, the "roundID" indicates the stage of the competition (eliminatory round, round of 16, quarter finals, semifinals, final);
- **seasonId**: indicates the season of the match;
- **status**: it can be "Played" (the match has officially finished), "Cancelled" (the match has been canceled for some reason), "Postponed" (the match has been postponed and no new date and time is available yet) or "Suspended" (the match has been suspended and no new date and time is available yet);
- **venue**: the stadium where the match was held (e.g., "Stadio Olimpico");
- **winner**: the identifier of the team which won the game, or 0 if the match ended with a draw;
- **wyId**: the identifier of the match, assigned by Wyscout;
- **teamsData**: it contains several subfields describing information about each team that is playing that match: such as lineup, bench composition, list of substitutions, coach and scores:
- **hasFormation**: it has value 0 if no formation (lineups and benches) is present, and 1 otherwise;
- **score**: the number of goals scored by the team during the match (not counting penalties);
- **scoreET**: the number of goals scored by the team during the match, including the extra time (not counting penalties);
- **scoreHT**: the number of goals scored by the team during the first half of the match;
- **scoreP**: the total number of goals scored by the team after the penalties;
- **side**: the team side in the match (it can be "home" or "away");
- **teamId**: the identifier of the team;
- **coachId**: the identifier of the team's coach;
- **bench**: the list of the team's players that started the match in the bench and some basic statistics about their performance during the match (goals, own goals, cards);
- **lineup**: the list of the team's players in the starting lineup and some basic statistics about their performance during the match (goals, own goals, cards);
- **substitutions**: the list of team's substitutions during the match, describing the players involved and the minute of the substitution.

In [ ]:
matches = json.load(open('matches_Italy.json'))

In [ ]:
a_match = matches[0]
match_id = a_match['wyId']
print(match_id)
a_match

## Loading an `event` document

An event refers to a ball touch and contains the following information:

- **eventId**: the identifier of the event's type. Each eventId is associated with an event name (see next point);
- **eventName**: tteamIdhe name of the event's type. There are seven types of events: pass, foul, shot, duel, free kick, offside and touch;
- **subEventId**: the identifier of the subevent's type. Each subEventId is associated with a subevent name (see next point);
- **subEventName**: the name of the subevent's type. Each event type is associated with a different set of subevent types;
- **tags**: a list of event tags, each one describes additional information about the event (e.g., accurate). Each event type is associated with a different set of tags;
- **eventSec**: the time when the event occurs (in seconds since the beginning of the current half of the match);
- **id**: a unique identifier of the event;
- **matchId**: the identifier of the match the event refers to. The identifier refers to the field "wyId" in the match dataset;
- **matchPeriod**: the period of the match. It can be "1H" (first half of the match), "2H" (second half of the match), "E1" (first extra time), "E2" (second extra time) or "P" (penalties time);
- **playerId**: the identifier of the player who generated the event. The identifier refers to the field "wyId" in a player dataset;
- **positions**: the origin and destination positions associated with the event. Each position is a pair of coordinates (x, y). The x and y coordinates are always in the range [0, 100] and indicate the percentage of the field from the perspective of the attacking team. In particular, the value of the x coordinate indicates the event's nearness (in percentage) to the opponent's goal, while the value of the y coordinates indicates the event's nearness (in percentage) to the right side of the field;
- **teamId**: the identifier of the player's team. The identifier refers to the field "wyId" in the team dataset.

In [ ]:
all_events = json.load(open('events_Italy.json'))
len(all_events)

In [ ]:
an_event = all_events[0]
an_event

### Load all the events of a specific match

In [ ]:
a_match_events = [event for event in all_events if event['matchId'] == match_id]
len(a_match_events)

## Visualize the match events

In [ ]:
pitch_layout = dict(hovermode='closest', autosize=False,
                                    width=825,
                                    height=600,
                                    plot_bgcolor='rgb(59,205,55)',
                                    xaxis={
                                        'range': [0, 100],
                                        'showgrid': False,
                                        'showticklabels': True,
                                    },
                                    yaxis={
                                        'range': [0, 100],
                                        'showgrid': False,
                                        'showticklabels': True,
                                    },
                                    shapes=[
                                        {
                                            'type': 'circle',
                                            'xref': 'x',
                                            'yref': 'y',
                                            'y0': 35,
                                            'x0': 40,
                                            'y1': 65,
                                            'x1': 60,
                                            'line': {
                                                'color': 'white',
                                            },
                                            
                                        },
                                     {
                                            'type': 'line',
                                            'xref': 'x',
                                            'yref': 'y',
                                            'y0': 35,
                                            'x0': 0,
                                            'y1': 35,
                                            'x1': 10,                                         
                                            'line': {
                                                'color': 'white',
                                            },
                                            
                                        },
                                     {
                                            'type': 'line',
                                            'xref': 'x',
                                            'yref': 'y',
                                            'y0': 35,
                                            'x0': 10,
                                            'y1': 65,
                                            'x1': 10,
                                            'line': {
                                                'color': 'white',
                                            }
                                     },
                                    {
                                        'type': 'line',
                                        'xref': 'x',
                                        'yref': 'y',
                                        'y0': 65,
                                        'x0': 10,
                                        'y1': 65,
                                        'x1': 0,
                                        'line': {
                                            'color': 'white',
                                        }
                                      },
                                    {
                                            'type': 'line',
                                            'xref': 'x',
                                            'yref': 'y',
                                            'y0': 35,
                                            'x0': 100,
                                            'y1': 35,
                                            'x1': 90,                                         
                                            'line': {
                                                'color': 'white',
                                            },
                                            
                                        },
                                     {
                                            'type': 'line',
                                            'xref': 'x',
                                            'yref': 'y',
                                            'y0': 35,
                                            'x0': 90,
                                            'y1': 65,
                                            'x1': 90,
                                            'line': {
                                                'color': 'white',
                                            }
                                     },
                                    {
                                        'type': 'line',
                                        'xref': 'x',
                                        'yref': 'y',
                                        'y0': 65,
                                        'x0': 90,
                                        'y1': 65,
                                        'x1': 100,
                                        'line': {
                                            'color': 'white',
                                        }
                                      },    
                                    {
                                        'type': 'line',
                                        'xref': 'x',
                                        'yref': 'y',
                                        'y0': 100,
                                        'x0': 50,
                                        'y1': 0,
                                        'x1': 50,
                                        'line': {
                                            'color': 'white',
                                        }
                                      }, 
                                    ]
)

In [ ]:
def visualize_events(a_match, a_match_events):
    """
    Visualize all the events of a match on the soccer pitch.
    
    Parameters
    ----------
    a_match: dict
        a dictionary containing the information of the match
        
    a_match_events: list
        a list of event dictionaries
    """
    team1, team2 = a_match['teamsData'].keys()

    # Create and style traces
    trace1 = go.Scatter(
        x = [event['positions'][0]['x'] for event in a_match_events if str(event['teamId']) == team1],
        y = [event['positions'][0]['y'] for event in a_match_events if str(event['teamId']) == team1],
        text = ['%s by player %s' %(event['eventName'], event['playerId']) for event in a_match_events if str(event['teamId']) == team1],
        mode = 'markers',
        name = team1,
        marker = dict(
            size = 5,
            color = 'red',
        )
    )

    trace2 = go.Scatter(
        x = [event['positions'][0]['x'] for event in a_match_events if str(event['teamId']) == team2],
        y = [event['positions'][0]['y'] for event in a_match_events if str(event['teamId']) == team2],
        text = ['%s by player %s' %(event['eventName'], event['playerId']) for event in a_match_events if str(event['teamId']) == team2],
        mode = 'markers',
        name = team2,
        marker = dict(
            size = 5,
            color = 'blue',
            symbol='square'
        )
    )

    fig = dict(data=[trace1, trace2], layout=pitch_layout)
    iplot(fig)

In [ ]:
visualize_events(a_match, a_match_events)

In [ ]:
def n_passes(a_match_events):
    """
    Compute the number of passes per player
    
    Parameters
    ----------
    a_match_events: list
        a list of event dictionaries
        
    Returns
    -------
    pandas DataFrame
    """
    player2n_passes = defaultdict(int)
    for event in a_match_events:
        player_id = event['playerId']
        event_type = event['eventName']
        if event_type == 'Pass':
            player2n_passes['id_%s' %player_id] += 1
    df = pd.DataFrame.from_dict(player2n_passes, orient='index').reset_index()
    df.columns = ['player_id', 'n_passes']
    df.player_id = df.player_id.astype(str)
    return df

def n_shots(a_match_events):
    """
    Compute the number of shots per player
    
    Parameters
    ----------
    a_match_events: list
        a list of event dictionaries
        
    Returns
    -------
    pandas DataFrame
    """
    player2n_shots = defaultdict(int)
    for event in a_match_events:
        player_id = event['playerId']
        event_type = event['eventName']
        if event_type == 'Shot':
            player2n_shots['id_%s' %player_id] += 1
    df = pd.DataFrame.from_dict(player2n_shots, orient='index').reset_index()
    df.columns = ['player_id', 'n_shots']
    df.player_id = df.player_id.astype(str)
    return df

def pass_accuracy(a_match_events):
    """
    Compute the pass accuracy of each player
    
    Parameters
    ----------
    a_match_events: list
        a list of event dictionaries
        
    Returns
    -------
    pandas DataFrame
    """
    player2n_passes = defaultdict(int)
    player2n_acc_passes = defaultdict(int)
    
    for event in a_match_events:
        player_id = event['playerId']
        event_type = event['eventName']
        tags = [tag['id'] for tag in event['tags']]
        if event_type == 'Pass':
            player2n_passes['id_%s' %player_id] += 1
            if 1801 in tags:
                player2n_acc_passes['id_%s' %player_id] += 1
    
    player2pass_accuracy = defaultdict(float)
    for player, passes in player2n_passes.items():
        player2pass_accuracy[player] = player2n_acc_passes[player] / passes
        
    df = pd.DataFrame.from_dict(player2pass_accuracy, orient='index').reset_index()
    df.columns = ['player_id', 'pass_accuracy']
    df.player_id = df.player_id.astype(str)
    return df
        
def shot_accuracy(a_match_events):
    """
    Compute the shot accuracy of each player
    
    Parameters
    ----------
    a_match_events: list
        a list of event dictionaries
        
    Returns
    -------
    pandas DataFrame
    """
    player2n_shots = defaultdict(int)
    player2n_acc_shots = defaultdict(int)
    
    for event in a_match_events:
        player_id = event['playerId']
        event_type = event['eventName']
        tags = [tag['id'] for tag in event['tags']]
        if event_type == 'Shot':
            player2n_shots['id_%s' %player_id] += 1
            if 1801 in tags:
                player2n_acc_shots['id_%s' %player_id] += 1
    
    player2shot_accuracy = defaultdict(float)
    for player, shots in player2n_shots.items():
        player2shot_accuracy[player] = player2n_acc_shots[player] / shots
        
    df = pd.DataFrame.from_dict(player2shot_accuracy, orient='index').reset_index()
    df.columns = ['player_id', 'shot_accuracy']
    df.player_id = df.player_id.astype(str)
    return df
    
def visualize_barchart(df, kind='n_passes'):
    """
    Visualize a quantity through a bar chart
    
    Parameters
    ----------
    df: pandas DataFrame
        a DataFrame containing the information to visualize
        
    kind: str
        the name of the column in the DataFrame that must be visualized
    """
    data = [go.Bar(
            x=df.player_id,
            y=df[kind]
    )]

    layout = go.Layout(
        xaxis=dict(
            autorange=True,
            showgrid=False,
            zeroline=True,
            showline=True,
            showticklabels=True,
            title='players'
        ),
        yaxis=dict(
            autorange=True,
            showgrid=False,
            zeroline=True,
            showline=True,
            showticklabels=True,
            title=kind
        )
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)

## Number of passes per player

In [ ]:
df_passes = n_passes(a_match_events)
visualize_barchart(df_passes, kind='n_passes')

## Number of shots per player

In [ ]:
df_shots = n_shots(a_match_events)
visualize_barchart(df_shots, kind='n_shots')

## Pass accuracy of each player

In [ ]:
df_pa = pass_accuracy(a_match_events)
visualize_barchart(df_pa, 'pass_accuracy')

## Shot accuracy of each player

In [ ]:
df_sa = shot_accuracy(a_match_events)
visualize_barchart(df_sa, 'shot_accuracy')